# Task: Model the dynamics of gender in intro CS


In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Import libraries


import sys
sys.path.append('tools/')

import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplt
import seaborn as sns
import tools
import inputData
   
pyplt.style.use('ggplot')

## Exploring the Data

Let's go ahead and read in the student dataset first.


In [3]:
dataset = inputData.preprocess()

In [4]:
dataset = dataset.query('gender == "Female" or gender == "Male"')
dataset = dataset.reset_index(drop=True)


print '{:d}{:20}{:d}{:20}'.format(len(dataset.query('gender == "Male"')), ' male students, and ',
                                  len(dataset.query('gender == "Female"')),
                                  ' female students consented to participate in this study')
                        

494 male students, and 388 female students consented to participate in this study


In [5]:
inputData.DESCR()

UC Berkeley Intro CS Student dataset

Notes
------
Data Set Characteristics:

Number of Instances:882

Attribute Information (in order):
Self reported attitudes about CS
- atcs_1 I like to use computer science to solve problems.
- atcs_2 I can learn to understand computing concepts.
- atcs_3 I can achieve good grades (C or better) in computing courses.
- atcs_4 I do not like using computer science to solve problems.
- atcs_5 I am confident that I can solve problems by using computation.
- atcs_6 The challenge of solving problems using computer science appeals to me.
- atcs_7 I am comfortable with learning computing concepts.
- atcs_8 I am confident about my abilities with regards to computer science.
- atcs_9 I do think I can learn to understand computing concepts.

Gendered belief about CS ability
- atcsgender_1 Women are less capable of success in CS than men.
- atcsgender_2 Women are smarter than men.
- atcsgender_3 Men have better math and science abilities than women.

Career driv

## Preprocess Data

### Preprocess feature columns
To prepare our data for classification, we need to devise a scheme to transform all features into numeric data. This dataset as several non-numeric columns that need converting. Many of them are simply `yes`/`no`, e.g. `prcs_2`. We can reasonably convert these into `1`/`0` (binary) values. For the columns whose values are `Nan`, we will convert these to `0`. 


**Note**: These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [ ]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['Yes', 'No'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            # e.g. 'reason' => 'reason_class_Interested' , 'reason_class_Other'
            col_data = pd.get_dummies(col_data, prefix=col)

        outX = outX.join(col_data)  # collect column(s) in output dataframe
        outX.fillna(0, inplace=True) # make sure all NaN <missing> values are set to 0

    return outX



In [ ]:
dataset = preprocess_features(dataset)
print "Processed feature columns ({}):- {}".format(len(dataset.columns), list(dataset.columns))


### Preprocess feature columns - Rename columns

There are some columns that have spaces in their names, these makes it difficult for the tree plotting algorithms that we will be using later to graph these features. As a result, we will change these spaces to hypens.

In [ ]:
dataset.rename(columns = {'grade_B or above':'grade_B_or_above'}, inplace = True)
dataset.rename(columns = {'grade_B or below':'grade_B_or_below'}, inplace = True)

### Scaling
Linearly scale each attribute to the range [0, 1].



In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_scaled = pd.DataFrame(scaler.fit_transform(dataset), columns=dataset.columns)
dataset = df_scaled
dataset.tail()


### Extract features and labels 

Extract feature (X) and target (y) columns

In [ ]:
target_col = dataset['gender_Female']  #  column is the target/label 
y = target_col  # corresponding targets/labels

print "\nLabel values:-"
print y.head()  

In [ ]:
X = dataset.drop(['gender_Female', 'gender_Male'], axis=1, inplace=False)

print "\nFeature values:-"
print X.head()  

In [ ]:
## Make sure we drop the target from the table, otherwise we will be including the labeled data

print "Feature column(s):-", list(X)

### Determine if classes are balanced
- As we can see the dataset is unbalanced, we have more males than females.




In [ ]:
y.plot.hist()

_= pyplt.xlabel('Value')
_= pyplt.title('Histogram of Target Class')
_= pyplt.legend(loc='upper center', shadow=True, fontsize='medium')
_= pyplt.yticks(np.arange(0, 1000, 100))
pyplt.savefig('report/figures/targetClass.pdf', dpi=100)


## Feature Transformation
We will use principal component analysis (PCA) to draw conclusions about the underlying structure of the data. Since using PCA on a dataset calculates the dimensions which best maximize variance, we will find which compound combinations of features best describe students.

In [ ]:
# TODO: Apply PCA by fitting the good data with the same number of dimensions as features
from sklearn.decomposition import PCA

pca_data = dataset.drop(['gender_Female', 'gender_Male'], axis=1, inplace=False)

pca = PCA(n_components=30)
pca.fit(pca_data)


In [ ]:
#Explained variance


fig, ax = pyplt.subplots()

_= pyplt.plot(np.cumsum(pca.explained_variance_ratio_))
_= pyplt.title('Explained Variance v. Number of Principal Components')
_= pyplt.xlabel('number of components')
_= pyplt.ylabel('cumulative explained variance')
_= pyplt.show()

fig.savefig('report/figures/explainedVariance.pdf', format='pdf', dpi=100)








In [ ]:

print("Projecting the input data on the eigenfaces orthonormal basis")
X_pca = pca.transform(X)

In [ ]:
X_pca = pd.DataFrame(X_pca)

In [ ]:
# First, decide how many training vs test samples you want
num_all = X.shape[0]  # same as len(student_data)
num_train = 662  # about 75% of the data
num_test = num_all - num_train

In [ ]:
num_train

In [ ]:
from sklearn import cross_validation



def shuffle_split_data(X, y):
    """ Shuffles and splits data into 75% training and 25% testing subsets,
        then returns the training and testing subsets. """
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, 
                                                                        train_size=num_train, random_state=42)
    
    
    


    # Return the training and testing data subsets
    return X_train, y_train, X_test, y_test

In [ ]:
try:
    X_train, y_train, X_test, y_test = shuffle_split_data(X, y)
    print "Successfully shuffled and split the data!"
except:
    print "Something went wrong with shuffling and splitting the data."


print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])


In [ ]:
X_pca_train, y_pca_train, X_pca_test, y_pca_test = shuffle_split_data(X_pca, y)
print "Training set: {} samples".format(X_pca_train.shape[0])
print "Test set: {} samples".format(X_pca_test.shape[0])

## Modeling and Validating




I have created four separate tranches to investigate the data:
- All features
- A reduced dataset of features
- PCA transformed features

## Train on all features

In [ ]:
X = X_train

seed = 342 # For reproducability
np.random.seed(seed)
folds = 50

In [ ]:

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import tree
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.learning_curve import validation_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score


models = {
          'XGBoost': XGBClassifier(),
          'DecisionTree': tree.DecisionTreeClassifier(),
          'SVC': svm.SVC(),
          'RandomForest': RandomForestClassifier()
         }


In [ ]:
for model_name, model in models.iteritems():
    print model_name
    kfold = StratifiedKFold(y_train, n_folds=folds, random_state=seed)
    results = cross_val_score(model, X, y_train, cv=kfold)
    print("Training data accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)), '\n'
   

### Predict 

In [ ]:
Features_test = X_test

for model_name, model in models.iteritems():
    print model_name
    # make predictions for test data
    model.fit(X, y_train)
    y_pred = model.predict(Features_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    print("Testing data prediction accuracy: %.2f%%" % (accuracy * 100.0)), '\n'

### Select XGBoost classifier
- Generate Features from XGBoost classifier 

In [ ]:
# plot feature importance
from xgboost import plot_importance

model = XGBClassifier()
model.fit(X, y_train)




plot_importance(model)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(14, 8)
fig.savefig('report/figures/feature_importance_xgb.pdf', dpi=100)

In [ ]:
g = xgb.to_graphviz(model, rankdir='LR')
g.render('report/figures/X_graph.gv', view=True)
    

## Train on a reduced set of features
- We will reduce the number of features used based on feature importance given by xgboost classifier

In [ ]:
# Extract feature (X) and target (y) columns

columnsToModel = [
    'atcsgender_2', 'clet_2', 'atcsjob_2', 'atct_6', 'prepared',
    'atcs_6', 'atct_5', 'atct_3', 'atcsjob_1', 'blg_1', 'atcsgender_3',
    'atct_7', 'atct_4', 'clet_1', 'atct_8', 'atcs_5', 'atcsgender_1',
    'cltrcmp_1', 'cltrcmp_2', 'blg_4']

X_all_Reduced = X_train[columnsToModel]


print "\nFeature values:-"
print X_all_Reduced.head()  

In [ ]:
model = XGBClassifier()
print "Training: ", model.__class__.__name__
kfold = StratifiedKFold(y_train, n_folds=folds, random_state=seed)
results = cross_val_score(model, X_all_Reduced, y_train, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)), '\n'



In [ ]:
Features_test = X_test[columnsToModel]



model.fit(X_all_Reduced, y_train)
y_pred = model.predict(Features_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Testing data prediction accuracy: %.2f%%" % (accuracy * 100.0)), '\n'

In [ ]:
model = XGBClassifier()
model.fit(X_all_Reduced, y_train)
g = xgb.to_graphviz(model, rankdir='LR')
g.render('report/figures/X_all_Reduced_graph.gv', view=True)

## Train on PCA data

In [ ]:
model = XGBClassifier()
print "Training: ", model.__class__.__name__
kfold = StratifiedKFold(y_pca_train, n_folds=folds, random_state=seed)
results = cross_val_score(model, X_pca_train, y_pca_train, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)), '\n'


In [ ]:
model.fit(X_pca_train, y_pca_train)
y_pred = model.predict(X_pca_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_pca_test, predictions)
print("Testing data prediction accuracy: %.2f%%" % (accuracy * 100.0)), '\n'

# Tune Estimator

### Ideal choices of parameters as taken from Xgboost With Python ebook

- Number of Trees (`n_estimators`) set to a fixed value between 100 and 1000, depending on the dataset size.
- Learning Rate (`learnin_rate`) simplified to the ratio: [2 to 10]/trees, depending on the trees number of trees.
- Row Sampling (`subsample`) grid searched values in the range [0.5, 0.75, 1.0].
- Column Sampling (`colsample` bytree and maybe colsample bylevel) grid searched values in the range [0.4, 0.6, 0.8, 1.0].
- Min Leaf Weight (`min_child_weight`) simplified to the ratio 3/rare_events , where rare events rare events is the percentage of rare event observations in the dataset.
- Tree Size (`max_depth`) grid searched values in the rage [4, 6, 8, 10].
- Min Split Gain (`gamma`) fixed with a value of zero.


In [ ]:
from __future__ import division


X = X_train

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import f1_score, make_scorer
from xgboost.sklearn import XGBClassifier



default_params = {
    'objective': 'binary:logistic',
    'max_depth': 1,
    'learning_rate': 0.3
}


n_estimators_range = np.linspace(1, 200, 10).astype('int') 

# Build a stratified shuffle object because of unbalanced data
folds = 50
ssscv = StratifiedShuffleSplit(y_train, folds, random_state=seed)


train_scores, test_scores = validation_curve(
    XGBClassifier(**default_params),
    X, y_train,
    param_name = 'n_estimators',
    param_range = n_estimators_range,
    cv=ssscv,
    scoring='accuracy'
)


 Show the validation curve plot

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

fig = plt.figure(figsize=(10, 6), dpi=100)

plt.title("Validation Curve with XGBoost (eta = 0.3)")
plt.xlabel("number of trees")
plt.ylabel("Accuracy")
plt.ylim(0.4, 1.1)

plt.plot(n_estimators_range,
             train_scores_mean,
             label="Training score",
             color="r")

plt.plot(n_estimators_range,
             test_scores_mean, 
             label="Cross-validation score",
             color="g")

plt.fill_between(n_estimators_range, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, 
                 alpha=0.2, color="r")

plt.fill_between(n_estimators_range,
                 test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std,
                 alpha=0.2, color="g")

plt.axhline(y=1, color='k', ls='dashed')

plt.legend(loc="best")
plt.show()

i = np.argmax(test_scores_mean)
print("Best cross-validation result ({0:.2f}) obtained for {1} trees".format(test_scores_mean[i], 
                                                                             n_estimators_range[i]))

fig.savefig('report/figures/varianceCurve_1.pdf', format='pdf')


Looking at the plot we can draw the following conclusions:
- training score keeps growing while adding new trees, but from a certain point CV score is fixed
- variance is lowest, and bias is high for less than 25 trees,
- we can see that the model is quite stable keeping variance fixed when increasing it's complexity   

We can assume that the trade-off for our model will be met at n_estimators = 50. The variance is still to big.


Tweak parameters
- To reduce a variance:  
    - each tree we will use 70% randomly chosen samples
    - 60% randomly chosen features   
    
- To decrease the bias (bigger accuracy):
    - Add an extra level to each tree


In [ ]:
default_params = {
    'objective': 'binary:logistic',
    'max_depth': 2, # changed
    'learning_rate': 0.3,
    'colsample_bytree': 0.6, # added
    'subsample': 0.7 # added
}

n_estimators_range = np.linspace(1, 200, 10).astype('int')

train_scores, test_scores = validation_curve(
    XGBClassifier(**default_params),
    X, y_train,
    param_name = 'n_estimators',
    param_range = n_estimators_range,
    cv=ssscv,
    scoring='accuracy'
)

In [ ]:

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

fig = plt.figure(figsize=(10, 6), dpi=100)

plt.title("Validation Curve with XGBoost (eta = 0.3)")
plt.xlabel("number of trees")
plt.ylabel("Accuracy")
plt.ylim(0.4, 1.1)
plt.plot(n_estimators_range,
             train_scores_mean,
             label="Training score",
             color="r")

plt.plot(n_estimators_range,
             test_scores_mean, 
             label="Cross-validation score",
             color="g")

plt.fill_between(n_estimators_range, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, 
                 alpha=0.2, color="r")

plt.fill_between(n_estimators_range,
                 test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std,
                 alpha=0.2, color="g")

plt.axhline(y=1, color='k', ls='dashed')

plt.legend(loc="best")
plt.show()

i = np.argmax(test_scores_mean)
print("Best cross-validation result ({0:.2f}) obtained for {1} trees".format(test_scores_mean[i], 
                                                                             n_estimators_range[i]))
fig.savefig('report/figures/varianceCurve_2.pdf', format='pdf')

In [ ]:
params_grid = {
            'learning_rate': [0.2222, 0.4444, 0.6666, 0.8888],
            'max_depth': [4, 6, 8, 10],
            'n_estimators': range(100, 1100, 100),
            'colsample_bytree': [0.6],
            'subsample':[0.7]
}

params_fixed = {
    'objective': 'binary:logistic',
    'silent': 1
}

In [ ]:
print '_'*20, 'Tuning XGBoost', '_'*20


grid = GridSearchCV(estimator=XGBClassifier(**params_fixed),
    param_grid=params_grid,
    cv=ssscv,
    scoring='accuracy')
grid.fit(X, y_train)


In [ ]:
print "Best accuracy obtained: {0}".format(grid.best_score_)
print "Parameters:"
for key, value in grid.best_params_.items():
    print "\t{}: {}".format(key, value)
    



In [ ]:
Features_test = X_test


model = grid.best_estimator_

# make predictions for test data
model.fit(X, y_train)
y_pred = model.predict(Features_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

## Save Model

In [ ]:
import pickle

# save model to file
pickle.dump(model, open("genderedCSExperience.pickle.dat", "wb"))

In [ ]:
model.fit(X, y_train)
g = xgb.to_graphviz(model, num_trees=2, rankdir='LR')
g.render('report/figures/Tuned_model_graph.gv', view=True)

In [ ]:
inputData.describeData(itemDimensions['atct'])